<a href="https://colab.research.google.com/github/ssv273/Recom_systems/blob/master/hw_webinar_4_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
!pip install implicit

     |████████████████████████████████| 1.1MB 2.7MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420270 sha256=f8b446daa94a8a9b75fa0bea03db177f045cc379595617ba3b76da49a3ba3dc9
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path_1 = '/content/drive/My Drive/Recomendation_systems/src'
module_path_2 = '/content/drive/My Drive/Recomendation_systems/'
if module_path_1 not in sys.path:
    sys.path.append(module_path_1)
if module_path_2 not in sys.path:
    sys.path.append(module_path_2)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [5]:
data = pd.read_csv('/content/drive/My Drive/Recomendation_systems/Data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
item_features = pd.read_csv('/content/drive/My Drive/Recomendation_systems/Data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [8]:
n_items_before = data_train['item_id'].nunique()
n_users_before = data_train['user_id'].nunique()

data_train = prefilter_items(data_train, item_features, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
n_users_after = data_train['user_id'].nunique()

print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))
print('Decreased # users from {} to {}'.format(n_users_before, n_users_after))

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Decreased # items from 86865 to 5001
Decreased # users from 2499 to 2497


-----

# Домашнее задание

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [0]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    data : pd.DataFrame
    item_features : pd.DataFrame
        данные по продажам и данные по признакам
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
        self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
        self.popularity_matrix = self.prepare_popularity_item_matrix(data, self.model)
        self.top_popular_items = self.popularity_recommendation(data)
        
     
    @staticmethod
    def prepare_matrix(data):
        
        # your_code
        user_item_matrix = pd.pivot_table(data, 
                                          index='user_id', columns='item_id', 
                                          values='quantity', # Можно пробоват ьдругие варианты
                                          aggfunc='count', 
                                          fill_value=0
                                        )

        user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix, K=1):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=K, num_threads=1)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=1, use_gpu=False):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads,
                                             use_gpu=use_gpu)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return model

    
    def prepare_popularity_item_matrix(self, data, model, N=20):
        """Подготавливает матрицу похожести N популярных у каждого юзера"""

        popularity = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        popularity.sort_values('quantity', ascending=False, inplace=True)

        popularity = popularity[popularity['item_id'] != 999999]

        popularity = popularity.groupby('user_id').head(N)

        popularity.sort_values('user_id', ascending=False, inplace=True)

        popularity['similar_item'] = popularity['item_id'].apply(lambda x: self.id_to_itemid[self.model.similar_items(self.itemid_to_id[x], N=2)[1][0]])

        popularity = popularity.groupby('user_id')['similar_item'].unique()

        return popularity

    @staticmethod
    def popularity_recommendation(data, n=50):
      """Топ-n популярных товаров"""
      
      popular = data.groupby('item_id')['sales_value'].sum().reset_index()
      popular.sort_values('sales_value', ascending=False, inplace=True)
      
      recs = popular.head(n).item_id
      
      return recs.tolist()[1:]

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # your_code
        
        res = list(self.popularity_matrix[user][:N].tolist())
        if len(res) < N:              # может случиться такая ситуация, когда юзер купил товаров меньше чем N, тогда его топ будет соответственно тоже меньше N
          lacks = N - len(res)        # и в таком случае мы просто добиваем популярными товарами
          res.extend(self.top_popular_items[:lacks])


        assert len(res) == N, 'Количество рекомендаций != {}, а = {} для юзера {}'.format(N, len(res), user)
        return res
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        # your_code
        sim_user = self.model.similar_users(self.userid_to_id[user], N=N+1)[1:]
        sim_user = np.array(sim_user)[:, 0].astype('int').tolist()

        res = []
        for j in sim_user[:N]:
          sim_user_top_item = self.own_recommender.recommend(j, csr_matrix(self.user_item_matrix).tocsr(), N=1)
          sim_user_top_item = self.id_to_itemid[sim_user_top_item[0][0]]
          res.append(sim_user_top_item)

        res = list(set(res))

        if len(res) < N:              # может случиться такая ситуация, когда в топе товаров у похожих юзеров находятся одинаковые товары
          lacks = N - len(res)        # и в таком случае мы просто добиваем популярными товарами
          res.extend(self.top_popular_items[:lacks])
        

        

        assert len(res) == N, 'Количество рекомендаций != {}, а = {} для юзера {}'.format(N, len(res), user)
        return res

In [10]:
%%time
main_recom = MainRecommender(data_train)


CPU times: user 18.6 s, sys: 10.7 s, total: 29.3 s
Wall time: 17 s


In [11]:
%%time
main_recom.get_similar_items_recommendation(16)

CPU times: user 204 µs, sys: 93 µs, total: 297 µs
Wall time: 305 µs


[8119135, 1016260, 1029743, 916122, 6533765]

In [12]:
%%time
main_recom.get_similar_users_recommendation(3)

CPU times: user 51.4 ms, sys: 45.8 ms, total: 97.3 ms
Wall time: 57.4 ms


[1069256, 1055403, 1057168, 958067, 894360]

In [13]:
res = main_recom.popularity_matrix.reset_index()
res.head()

,user_id,similar_item
0,1,"[952163, 1011459, 1088462, 12301839, 9297615, ..."
1,2,"[819978, 880888, 9416729, 12988031, 9837501, 8..."
2,3,"[1108296, 5568378, 1132178, 951590, 9677126, 9..."
3,4,"[1017195, 1105301, 10150194, 1021324, 951590, ..."
4,5,"[12262778, 1105301, 838882, 946839, 5568378, 1..."


In [0]:
res['top5_sim_items'] = res['user_id'].apply(lambda x: main_recom.get_similar_items_recommendation(x))

In [15]:
res.head()

,user_id,similar_item,top5_sim_items
0,1,"[952163, 1011459, 1088462, 12301839, 9297615, ...","[952163, 1011459, 1088462, 12301839, 9297615]"
1,2,"[819978, 880888, 9416729, 12988031, 9837501, 8...","[819978, 880888, 9416729, 12988031, 9837501]"
2,3,"[1108296, 5568378, 1132178, 951590, 9677126, 9...","[1108296, 5568378, 1132178, 951590, 9677126]"
3,4,"[1017195, 1105301, 10150194, 1021324, 951590, ...","[1017195, 1105301, 10150194, 1021324, 951590]"
4,5,"[12262778, 1105301, 838882, 946839, 5568378, 1...","[12262778, 1105301, 838882, 946839, 5568378]"


In [16]:
%%time
res['top5_sim_users'] = res['user_id'].apply(lambda x: main_recom.get_similar_users_recommendation(x))

CPU times: user 2min, sys: 1min 27s, total: 3min 27s
Wall time: 1min 45s


In [17]:
res.head()

,user_id,similar_item,top5_sim_items,top5_sim_users
0,1,"[952163, 1011459, 1088462, 12301839, 9297615, ...","[952163, 1011459, 1088462, 12301839, 9297615]","[948640, 7169090, 896666, 924667, 1036093]"
1,2,"[819978, 880888, 9416729, 12988031, 9837501, 8...","[819978, 880888, 9416729, 12988031, 9837501]","[1069256, 949257, 958067, 896666, 977374]"
2,3,"[1108296, 5568378, 1132178, 951590, 9677126, 9...","[1108296, 5568378, 1132178, 951590, 9677126]","[1069256, 1055403, 1057168, 958067, 894360]"
3,4,"[1017195, 1105301, 10150194, 1021324, 951590, ...","[1017195, 1105301, 10150194, 1021324, 951590]","[1117219, 939789, 872826, 6552318, 10341855]"
4,5,"[12262778, 1105301, 838882, 946839, 5568378, 1...","[12262778, 1105301, 838882, 946839, 5568378]","[948640, 977927, 902377, 5707857, 894360]"


----

Проверка, что все работает

In [0]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender as MR

In [20]:
mod = MR(data_train)

Экземпляр класса успешно создался из импортированного модуля